# Задание 1.

Основные тезисы:

1) Под условие попадает примерно вся западная половина районов Москвы, некоторые города и поселки городского типа Московской Области;

2) Около 7.5 миллионов человек могут добраться до офиса Tele-2 за час с помощью общественного транспорта.

Данное задание, на мой взгляд, выполнено не идеально. Первоначально я хотел построить соответствующую изохрону, импортировав все требуемые данные - остановки метро, общественного транспорта и тд с помощью SQL и дальнейшей работы с ними. Однако после двух дней, затраченных на такое решение без особых результатов, я решил решить задание менее качественно, но более надежно, с помощью библиотек arcgis и geopy в Python.

In [1]:
# Приступим! Импортируем необходимую библиотеку для работы с геоданными.
from geopy import Nominatim as nm

# Загружаем библиотеки numpy и pandas для визуализации точек с помощью arcgis.
import numpy as np
import pandas as pd

# Импортируем библиотеку для визуализации результатов на карте Москвы.
import arcgis
from arcgis.gis import GIS

In [2]:
# Создаем объект geo соответствующего класса для работы с координатами и геолокацией.
geo = nm()

/anaconda3/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [3]:
# Переменной location присваиваем адрес требуемого здания (офиса Теле-2).
location = geo.geocode("Москва, Киевское шоссе, 6/1")

In [4]:
# Проверим, правильное ли значение было присовено переменной.
location.address

'Киевское шоссе, поселение Московский, Новомосковский административный округ, Москва, ЦФО, 119027, РФ'

In [5]:
# Ошибка! Попробуем немного изменить форму записи.
location = geo.geocode("вл6 с1, 22 км Киевского шоссе, Москва")

In [6]:
# Вновь проверяем.
location.address

'Макет Comcity, вл6 с1 кА4, 22 км Киевского шоссе, поселение Московский, Новомосковский административный округ, Москва, ЦФО, 119606, РФ'

In [7]:
# То, что нужно! Выведем общую информацию о данном объекте.
location.raw

{'place_id': '52324366',
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'node',
 'osm_id': '4119212089',
 'boundingbox': ['55.6351155', '55.6352155', '37.4313936', '37.4314936'],
 'lat': '55.6351655',
 'lon': '37.4314436',
 'display_name': 'Макет Comcity, вл6 с1 кА4, 22 км Киевского шоссе, поселение Московский, Новомосковский административный округ, Москва, ЦФО, 119606, РФ',
 'class': 'tourism',
 'type': 'attraction',
 'importance': 0.711,
 'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_point_of_interest.p.20.png'}

Из этих данных нам существенны долгота, longitude, равная 37.4314436, и широта, latitude, которая, в свою очередь, равна 55.6351655.

Источники из интернета сообщают, что средняя скорость передвижения на общественном транспорте, будь то метро или наземный транспорт, составляет 41.5 км/ч с учетом остановок.

Путь до офиса состоит из дороги до общественного транспорта, непосредственной поездки на транспорте и дороги от остановки до офиса. Дорога до общественного транспорта плюс его ожидание (или спуск в метро) в среднем равна около 10 минутам. Путь от метро Румянцево или от троллейбусной/автобусной оставноки до офиса занимает по моему скромному опыту также порядка 10 минут. Соответственно, поездка на транспорте  должна занимать не более $60 - 10 - 10 = 40$ минут = $\frac{2}{3}$ часа. Со средней скоростью в 41.5 км/ч имеем $41.5 \cdot \frac{2}{3} \approx 27.6667$ километров.

Переведем данные единицы измерения в нужные нам: в одном градусе широты 111.134861111 километров. Разделив единицу на это число, получим количество градусов широты в 1 км. 1 / 111.134861111 = 0.00899807666. Аналогичным образом получим, что так как в одном градусе долготы 111.321377778 км, то в одном километре 0.008983000569 градусов. Таким образом, если разница между начальными и конечными координатами в $a$ градусов по долготе и $b$ градусов по широте, то расстояние в километрах равно $\sqrt{(\frac{a}{0.0089830})^2 + (\frac{b}{0.00899807})^2}$.

Итак, нас устраивают все районы, которые находятся на расстоянии 27.6667 километров. При переводе в градусы получим $\sqrt{(\frac{a}{0.0089830})^2 + (\frac{b}{0.00899807})^2} \leq 27.6667$. Возводя в квадрат обе части неравенства, получаем $(\frac{a}{0.0089830})^2 + (\frac{b}{0.00899807})^2 \leq 765.4445$. Теперь поделим стороны неравенства на 765.4445. $(\frac{a}{0.2485296})^2 + (\frac{b}{0.24894678})^2 \leq 1$. Данное уравнение характеризует элипс, включая всю его внутреннюю часть, с центром в исходной точке (координаты офиса Теле-2, как уже было выяснено, равны (37.4314436, 55.6351655) (долгота, широта)), у которого большая ось равна $2 \cdot 0.2485296 = 0.4970592$, а малая - $0.24894678 \cdot 2 = 0.49789356$. Нарисуем такое геометрическое место точек на карте.



In [8]:
# Для визуализации карты мы уже импортировали библиотеку arcgis. Заходим в аккаунт на сайте (логин и пароль указаны),
# создаем соответствующую переменную для работы.
gis = GIS(url='http://arcgis.com', username='Test_Akim', password='Testakim0')

In [9]:
# Посмотрим на Comcity с помощью данного модуля.
hello_map = gis.map("Москва, Comcity", zoomlevel = 16)

In [10]:
hello_map

MapView(layout=Layout(height='400px', width='100%'), zoom=16.0)

Отлично! Теперь требуется расставить крайние точки так, чтобы полученный ранее элипс охватывал все удовлетворяющие условию районы.

In [11]:
# Создаем список координат точек элипса по долготе. Соответствующие им координаты по широте выведем с помощью формулы элипса.
lon = np.arange(-0.2485296, 0.2485296, 10**(-2))

In [12]:
# Создадим два списка. В первый, lat_pos, поместим координаты точек элипса по широте, большие нуля. Соответственно,
# в другой список - меньшие нуля. 
lat_pos = [(1 - (i / 0.2485296)**2)**(1/2)*0.2485296 for i in lon]
lat_neg = [-i for i in lat_pos]

In [13]:
# Теперь инициализируем два списка: в первом лежат координаты точек, у которых изменение по широте "положительное",
# то есть к значению широты Comcity мы прибавляли какое-то значение. У координат точек из второго списка
# изменение по широте "отрицательное".
geom_pos = [(lon[i] + 37.4314436, lat_pos[i] + 55.6351655) for i in range(len(lon))]
geom_neg = [(lon[i] + 37.4314436, lat_neg[i] + 55.6351655) for i in range(len(lon))]

In [14]:
# Для каждого списка создадим DataFrame, который будет состоять из трех столбцов: долгота, широта и фактический адрес точки.
# Адрес будем получать с помощью геокода (geocode), зная координаты объекта. DataFrame нужна для визуализации данных точек на карте.
df_pos = {'Longitude': [j[0] for j in geom_pos], 'Latitude': [j[1] for j in geom_pos]}
df_pos = pd.DataFrame(df_pos)
df_pos['Address'] = [geo.geocode(
    '{0}, {1}'.format(df_pos.Latitude[i], df_pos.Longitude[i])).address for i in range(len(df_pos))]
df_pos

,Longitude,Latitude,Address
0,37.182914,55.635165,"Северный объезд Одинцова, сельское поселение Ж..."
1,37.192914,55.704955,"сельское поселение Горское, Одинцовский район,..."
2,37.202914,55.732844,"Государственная резиденция «Ново-Огарёво», Руб..."
3,37.212914,55.753537,"Александровка, городской округ Красногорск, Мо..."
4,37.222914,55.770378,"46К-9040, Ильинское-Усово, городской округ Кра..."
5,37.232914,55.784675,"46К-9030, Поздняково, городской округ Красного..."
6,37.242914,55.797102,"Изумрудные холмы, городской округ Красногорск,..."
7,37.252914,55.808065,"Гореносово, Изумрудные холмы, дачного хозяйств..."
8,37.262914,55.817826,"7, улица Огарёва, Гореносово, Опалиха, Красног..."
9,37.272914,55.826569,"Опалиховский лесопарк, Светлая улица, Опалиха,..."


In [15]:
# Аналогичная операция для точек с отрицательным изменением широты.
df_neg = {'Longitude': [j[0] for j in geom_neg], 'Latitude': [j[1] for j in geom_neg]}
df_neg = pd.DataFrame(df_neg)
df_neg['Address'] = [geo.geocode(
    '{0}, {1}'.format(df_neg.Latitude[i], df_neg.Longitude[i])).address for i in range(len(df_neg))]
df_neg

,Longitude,Latitude,Address
0,37.182914,55.635165,"Северный объезд Одинцова, сельское поселение Ж..."
1,37.192914,55.565376,"Киевское шоссе, поселение Марушкинское, Новомо..."
2,37.202914,55.537487,"улица Александра Печерского, поселение Первома..."
3,37.212914,55.516794,"46Н-06106, поселение Первомайское, Троицкий ад..."
4,37.222914,55.499953,"Ново-Спасское, поселение Первомайское, Троицки..."
5,37.232914,55.485656,"поселение Первомайское, Троицкий административ..."
6,37.242914,55.473229,"поселение Первомайское, Троицкий административ..."
7,37.252914,55.462266,"Малыгино, Троицкий административный округ, Мос..."
8,37.262914,55.452505,"поселение Краснопахорское, Троицкий администра..."
9,37.272914,55.443762,"Былово, Троицкий административный округ, Москв..."


In [16]:
# Загружаем каждую DataFrame в наш объект gis.
tick_1 = gis.content.import_data(df_pos, address_fields={"Address": "Address", "Longitude": "Longitude", "Latitude": "Latitude"})
tick_2 = gis.content.import_data(df_neg, address_fields={"Address": "Address", "Longitude": "Longitude", "Latitude": "Latitude"})

In [17]:
# Визуализируем! Настроим формат карты, размер, и добавим новые точки. Для того, чтобы увидеть на карте отмеченные точки,
# нужно нажать на иконку, на которой расположен монитор и земной шар возле "+" в левом верхнем углу карты.
new = gis.map('Москва', zoomlevel=9)
new.basemap = 'osm'
new.height = '600px'
new.add_layer(tick_1)
new.add_layer(tick_2)
new

MapView(basemap='osm', layout=Layout(height='400px', width='100%'), zoom=9.0)

Отлично! Худо-бедно у нас получилось визуализировать районы, из которых можно добратсья до офиса Tele-2 на общественном транспорте, затратив на поездку не более часа. Безусловно, данный метод был далек от идеального: он не учитывал, что, например, если человек живет возле красной ветки, пусть даже возле последних ее станций (Бульвар Рокоссовского и тд), он тоже сможет добраться до офиса за час, несмотря на большое расстояние; в свою очередь, жителю Подольска, который попал внутрь нашей изохроны, будет требоваться немного больше: яндекс навигатор, например, оценивает непосредственно время поездки, равное примерно часу. Соответственно, с учетом всех затрат житель Подольска сможет добраться до офиса примерно за час двадцать - час тридцать. Однако данная изохрона, имеющая форму элипса, и показывающая данную категорию людей вполне неплохо, позволяет оценить среднее количество населения, имеющих данную возможность. Изохрона делит площадь Москвы примерно пополам. Соответственно, так как население Москвы оценивается на 2018 год как 12 млн людей, 6 млн жителей Мосеквы имеет возможность добраться до офиса Tele-2 в течение часа посредством общественного транспорта. Общая численность населения городов и поселков городского типа Московской области, удовлетворяющих заданному условию, равна еще примерно полутора миллионам. Итоговая численность населения составляет 7.5 миллионов человек. Пересечение изобары с районами Москвы хорошо видно на карте: почти вся западная половина райнонов Москвы удовлетворяет условию.

In [18]:
gis.map('Queen Mary', )

MapView(layout=Layout(height='400px', width='100%'))